# Исследование надёжности заёмщиков

Заказчик исследования — кредитный отдел банка.

Результаты исследования будут учтены при построении модели кредитного скоринга.

**Цель исследования** — проверка двух гипотез:
1. Семейное положение клиента влияет на факт погашения кредита в срок.
2. Наличие детей у клиента влияет на факт погашения кредита в срок.

***Дополнительно нужно проверить***:
1. Есть ли зависимость между уровнем дохода и возвратом кредита в срок.
2. Как разные цели кредита влияют на его возврат в срок.

**Ход исследования**

Входные данные от банка — статистика о платёжеспособности клиентов в файле`data.csv`. 
Перед проверкой гипотез понадобится обзор данных для проверки их качества.

Данные будут проверены на ошибки, а также будет дана оценка их влияния на исследование. 
Затем, на этапе предобработки будет рассмотрена возможность исправить самые критичные ошибки данных.
 
Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.



## Обзор данных

Составляю первое представление о статистике платёжеспособности клиентов.

In [1]:
import pandas as pd
try:
    df = pd.read_csv('data.csv')  # Локальный путь
except:
    df = pd.read_csv('/datasets/data.csv') 

Вывожу на экран первые десять строк таблицы:

In [2]:
df.head(10) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Получаю общую информацию о таблице:

In [3]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итак, в таблице одиннадцать столбцов. В столбцах три типа данных — `int64`, `float64`, `object`.


Согласно документации к данным:

* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита.

На первый взгляд названия колонок соответствуют стилю написания названий колонок. Далее проверю их на предмет отсутствия пробелов.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

В столбце общего трудового стажа в днях `days_employed` присутствуют отрицательные значения. Поскольку данные указанного столбца не будут использоваться в анализе, оставлю их отрицательными.

**Выводы**

В каждой строке таблицы — сведения о клиенте. Основная часть колонок описывает личные данные клиента: количество детей, трудовой стаж, возраст, уровень образования, семейное положение, пол,тип занятости и ежемесячный доход. Остальные данные рассказывают о наличии задолженностей по возврату кредитов и  о цели получения кредита. 

Предварительно можно утверждать, что данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а также отрицательные значения.

Для проверки гипотез и ответа на дополнительные вопросы будут использоваться данные шести столбцов:

* `children`
* `family_status`
* `family_status_id`
* `debt`
* `total_income`
* `purpose`

Чтобы двигаться дальше, нужно устранить проблемы в данных указанных столбцов.

## Предобработка данных
Проверяю отсутствие пробелов в заголовках столбцов, исключаю пропуски в данных. Проверяю данные на дубликаты.

### Стиль заголовков
Вывожу на экран названия столбцов:

In [4]:
df.columns 

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

Пробелов в названии столбцов нет.

### Пропуски значений
Сначала посчитаю, сколько в таблице пропущенных значений:

In [5]:
display(df.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В необходимом для исследования столбце `total_income` выявлены пропуски. 
Причиной этому могут быть сбои в выгрузке данных, ошибки при вводе данных.
В идеале на практике было бы правильно установить причину пропусков и восстановить данные. 
Поскольку эта возможность отсутствует, заполню пропущенные значения значением медианы ежемесячного дохода, используя метод median().

In [6]:
total_income_median = df['total_income'].median()    

# заполнение пропущенных значений значением медианы
df['total_income'] = df['total_income'].fillna(total_income_median)   

Еще раз считаю пропуски в таблице, чтобы убедиться, что в столбце `total_income` их не осталось. 

In [7]:
display(df.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

### Изменение типа данных

В столбце `total_income` изменяю тип данных с вещественного на целочисленный с использованием метода astype().

In [8]:
df['total_income'].astype('int64') 

0        253875
1        112080
2        145885
3        267628
4        158616
          ...  
21520    224791
21521    155999
21522     89672
21523    244093
21524     82047
Name: total_income, Length: 21525, dtype: int64

### Дубликаты

***Явные дубликаты***

Считаю явные дубликаты в таблице:

In [9]:
print(df.duplicated().sum())  

54


Дубликаты присутствуют. Причинами возникновения дубликатов могут быть некорректное соединение данных из разных источников в один, ошибки пользователей при занесении информации. Удаляю явные дубликаты:

In [10]:
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
df = df.drop_duplicates().reset_index(drop=True) 

Ещё раз считаю явные дубликаты в таблице, чтобы убедиться, что полностью от них избавилась:

In [11]:
print(df.duplicated().sum()) 

0


***Неявные дубликаты***

Теперь необходимо проверить значения в колонке `family_status` на наличие/отсутствие неявных дубликатов. 
Для этого выведу на экран список уникальных названий семейного положения:

In [12]:
df['family_status'].unique() 

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Всего 5 видов семейного положения, каждый представлен в единственном варианте, соответственно, дубликатов нет.

***Лемматизация***

Проверю значения в колонке `purpose` на наличие/отсутствие неявных дубликатов. 
Для этого выведу на экран список уникальных названий целей получения кредита:

In [13]:
df['purpose'].unique() 

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Из полученного списка видно, что одинаковые цели формулируются по-разному. Необходимо выделить леммы, чтобы можно было значения столбца объединить в категории в зависимости от целей получения кредита. В следующем разделе проекта `Категоризация данных` при категоризации будет использована лемматизация значений.

**Выводы**

Предобработка нарушений в стиле заголовков не обнаружила.

Предобработка обнаружила следующие проблемы в данных:
пропущенные значения,
дубликаты — явные и неявные.

Явные дубликаты удалены. 
Неявные дубликаты значений будут категоризированы с помощью лемматизации в следующем разделе проекта.

Пропущенные значения заменены на медианные. 

Теперь можно перейти к категоризации данных.

### Категоризация данных

***Количество детей в семье***

Для исследования необходимо значения столбца `children` разделить на две категории: есть дети и нет детей.
Сначала проверю уникальные значения столбца `children` и их количество.

In [14]:
df['children'].value_counts() 

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Обнаружено наличие отрицательных значений (- 1) и подозрительное количество (76) значений двадцати детей. 
Значение -1 ошибочное. Можно допустить, что возникла ошибка при выгрузке данных. Учитывая, что таких значений очень мало (всего 47), буду считать это значение за 0. 41 от 14107 составляет всего 0.3%.

Количество детей 20 у 76ти значений также говорит об ошибке. Однако если предположить, что в большей части этих ячеек указано больше детей, чем есть на самом деле, это никак не повлияет на ход исследования, поскольку нам важен факт наличия/отсутствия детей.

Напишу функцию, которая категоризирует значения стобца.

In [15]:
def create_category_children(data):

    if data == 0:
        return 'нет детей'
    return 'есть дети'
  
df ['children_category'] = df['children'].apply(create_category_children)

In [16]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_category
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,есть дети
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,есть дети
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,нет детей
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,есть дети
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,нет детей


***Ежемесячный доход***

Для исследования необходимо разделить на категории значения столбца `total_income`. 
Для начала посмотрю минимальное и максимальное значения.

In [17]:
df['total_income'].min()

20667.26379327158

In [18]:
df['total_income'].max()

2265604.028722744

Выделю 4 категории ежемесячного дохода:

* очень низкий (менее 60000)
* низкий (от 60000 до 100000)
* средний (от 100000 до 150000)
* высокий (более 150000)

Создам функцию для категоризации и добавлю столбец с данными по категориям.

In [19]:
def create_category_total_income(data):

    if data < 60000:
        return 'очень низкий'
    if 60000 <=data < 100000:
        return 'низкий'
    if 100000 <=data < 150000:
        return 'средний'
    return 'высокий'
      
df['total_income_category'] = df['total_income'].apply(create_category_total_income)

In [20]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_category,total_income_category
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,есть дети,высокий
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,есть дети,средний
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,нет детей,средний
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,есть дети,высокий
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,нет детей,высокий


***Цели получения кредита***

Напишу функцию, которая с помощью лемматизации разобьет значения столбца `purpose` на следующие категории:

* `автомобиль`
* `строительство`
* `ремонт`
* `недвижимость`
* `образование`
* `свадьба`

Выбор сделан с учетом:
 - составленного списка уникальных значений столбца `purpose` в предыдущем разделе проекта
 - связан с тем, что в банках для получения кредита заемщику необходимо собрать различный пакет документов в зависимости от целей кредита. К примеру, покупка недвижимости и ремонт недвижимости абсолютно разные категории. При покупке недвижимости в том числе предоставляются документы на приобретаемый объект (выписка из ЕГРН, тех.паспорт), при ремонте - договор с компанией, которая будет производить ремонт недвижимости.

In [21]:
# получение лемматизатора для слов на русском языке
from pymystem3 import Mystem 
m = Mystem() 

# создание функции для категоризации значений столбца 'purpose'    
def create_category_purpose(row): 
    # лемматизация значений столбца 'purpose'
    lem_purpose = m.lemmatize(row['purpose']) 
    if 'автомобиль' in lem_purpose:
        return 'автомобиль'
    if 'строительство' in lem_purpose:
        return 'строительство'
    if 'ремонт' in lem_purpose:
        return 'ремонт'
    if ('жилье' in lem_purpose) or ('недвижимость' in lem_purpose):
        return 'недвижимость'
    if 'образование' in lem_purpose:
        return 'образование'
    if 'свадьба' in lem_purpose:
        return 'свадьба'

# добавление в таблицу df столбца с категориями значений столбца 'purpose'
df ['purpose_category'] = df.apply(create_category_purpose, axis = 1) 


Проверю таблицу на наличие нового столбца `purpose_category` и значения в нем.

In [22]:
df.head(10) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_category,total_income_category,purpose_category
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,есть дети,высокий,недвижимость
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,есть дети,средний,автомобиль
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,нет детей,средний,недвижимость
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,есть дети,высокий,образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,нет детей,высокий,свадьба
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,нет детей,высокий,недвижимость
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,нет детей,высокий,недвижимость
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,нет детей,средний,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,есть дети,низкий,свадьба
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,нет детей,средний,недвижимость


Столбец `purpose_category` есть, значения категорий отражаются.
Проверю уникальные значения столбца `purpose_category` и их количество.

In [23]:
df['purpose_category'].value_counts() 

недвижимость     8328
автомобиль       4308
образование      4014
свадьба          2335
строительство    1879
ремонт            607
Name: purpose_category, dtype: int64

Все 6 категорий целей получения кредита присутствуют.

**Выводы**

Значения количества детей в семье разделены на две категории в соответствии с условиями гипотезы.
Значения ежемесячного доходы разделены на четыре категории.
Цели получения кредита разбиты на шесть категорий.

Для завершения предобработки осталось уточнить значения двух столбцов.

### Уточнение значений 

***Наличие задолженности по возврату кредита***

Проверю значения столбца `debt`. Для этого выведу уникальные значения и их количество.

In [24]:
df['debt'].value_counts()

0    19730
1     1741
Name: debt, dtype: int64

Согласно логике значение 0 должно соответствовать отсутствию задолженности, а значение 1 - наличию задолженности. 
Однако в этом необходимо убедиться. Изменю значения столбца на булевые и подсчитаю их.

In [25]:
df['debt'].astype('bool').value_counts()

False    19730
True      1741
Name: debt, dtype: int64

Все верно:
0 - отсутствие задолженности
1 - наличие задолженности

***Семейное положение***

Нужно проверить соотвествие значений столбца `family_status` значениям столбца `family_status_id`.
Для этого выведу уникальные значения столбцов и их количество.

In [26]:
df['family_status'].value_counts()

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [27]:
df['family_status_id'].value_counts()

0    12344
1     4163
4     2810
3     1195
2      959
Name: family_status_id, dtype: int64

Количество уникальных значений в двух столбцах совпадает. 
Соответственно, уникальные значения столбца `family_status_id` значат:
0 - женат / замужем
1 - гражданский брак
2 - вдовец / вдова
3 - в разводе
4 - гражданский брак

**Выводы**

Значения столбца с данными о наличии задолженности уточнены.

Значение семейного статуса и индекса семейного статуса сопоставлены. Данные предоставлены корректные.

Теперь можно перейти к проверке гипотез.

## Проверка гипотез

### Семейное положение клиента влияет на факт погашения кредита в срок

Первая гипотеза утверждает, что семейное положение клиента влияет на факт погашения кредита в срок. Для проверки гипотезы составлю сводную таблицу и отсортирую ее по убыванию значения процентного отношения задолженников ко всем заемщикам:

- индексом будет являться статус семейного положения
- столбцы: факт наличия/отсутствия задолженности по кредиту и процентное отношение задолженников ко всем заемщикам
- значения: количество значений любого столбца таблицы за исключением уже использованных


In [28]:
# формирование сводной таблицы
data_pivot = df.pivot_table(
    index = 'family_status', 
    columns = 'debt',
    values = 'family_status_id', 
    aggfunc = 'count'
)
# добавление столбца с процентым отношением
data_pivot['debt_percent'] = data_pivot[1]/(data_pivot[0]+data_pivot[1])*100 
# сортировка таблицы по столбцу debt_percent по убыванию
data_pivot = data_pivot.sort_values('debt_percent', ascending=False)  
data_pivot

debt,0,1,debt_percent
family_status,,,
Не женат / не замужем,2536,274,9.750890
гражданский брак,3775,388,9.320202
женат / замужем,11413,931,7.542126
в разводе,1110,85,7.112971
вдовец / вдова,896,63,6.569343


Согласно полученным данным серьезнее относятся к своевременному внесению средств в счет погашения кредита заемщики, состоящие в официальном браке, а также побывавшие в нем ранее (разведены, вдовцы). Причем последние более ответственные, особенно те, кто потерял свои вторые половины по причине смерти. Самый большой процент задолженности у несемейных заемщиков.

**Выводы**

Семейное положение действительно влияет на факт погашения задолженности в срок. Гипотеза подтвердилась.

### Наличие детей у клиента влияет на факт погашения кредита в срок ###

Вторая гипотеза утверждает, что наличие детей у клиента влияет на факт погашения кредита в срок. Для ее проверки составлю сводную таблицу:

- индексом будет являться категория наличия/отсутствия в семье детей
- столбцы: факт наличия/отсутствия задолженности по кредиту и процентное отношение задолженников ко всем заемщикам
- значения: количество любого столбца таблицы за исключением уже использованных

In [29]:
data_pivot = df.pivot_table(
    index = 'children_category',
    columns = 'debt',
    values = 'children', 
    aggfunc = 'count'
) 
# добавление столбца с процентым отношением
data_pivot['debt_percent'] = data_pivot[1]/(data_pivot[0]+data_pivot[1])*100 
data_pivot

debt,0,1,debt_percent
children_category,,,
есть дети,6686,678,9.206953
нет детей,13044,1063,7.535266


Согласно полученным данным семьи без детей допускают меньше просрочек по погашению кредитов.

**Выводы**

Наличие детей в семье действительно влияет на факт погашения задолженности в срок. Гипотеза подтвердилась.

#### Зависимость между уровнем дохода и возвратом кредита в срок

В данном исследовании также нужно выяснить, существует ли зависимость между уровнем дохода и возвратом кредита в срок.
Для этого составлю сводную таблицу и отсортирую ее по убыванию значения процентного отношения задолженников ко всем заемщикам:

- индексом будет являться категория ежемесячного дохода
- столбцы: факт наличия/отсутствия задолженности по кредиту и процентное отношение задолженников ко всем заемщикам
- значения: количество любого столбца таблицы за исключением уже использованных

In [30]:
data_pivot = df.pivot_table(
    index = 'total_income_category', 
    columns = 'debt',
    values = 'children', 
    aggfunc = 'count'
) 
data_pivot['debt_percent'] = data_pivot[1]/(data_pivot[0]+data_pivot[1])*100 
data_pivot = data_pivot.sort_values('debt_percent', ascending=False) 
data_pivot

debt,0,1,debt_percent
total_income_category,,,
средний,7163,661,8.448364
низкий,3352,305,8.340170
высокий,8458,726,7.905052
очень низкий,757,49,6.079404


Заемщики с очень низким доходом имеют самый низкий уровень задолженности. Самый высокий уровень задолженности у заемщиков со средним уровнем дохода. Однако он не намного выше, чем у заемщиков с низким и высоким уровнем дохода.

**Выводы**

Явная зависимость между уровнем дохода и возвратом кредита в срок прослеживается только в сравнении с очень низким уровнем дохода и иным доходом.

#### Влияние цели кредита на его возврат в срок

Также в данном исследовании также нужно выяснить, влияет ли цель получения кредита на его возврат в срок.
Для этого составлю сводную таблицу и отсортирую ее по убыванию значения процентного отношения задолженников ко всем заемщикам:

- индексом будет являться категория цели кредита
- столбцы: факт наличия/отсутствия задолженности по кредиту и процентное отношение задолженников ко всем заемщикам
- значения: количество любого столбца таблицы за исключением уже использованных

In [31]:
data_pivot = df.pivot_table(
    index = 'purpose_category', 
    columns = 'debt',
    values = 'family_status_id', 
    aggfunc = 'count'
) 
data_pivot['debt_percent'] = data_pivot[1]/(data_pivot[0]+data_pivot[1])*100 
data_pivot = data_pivot.sort_values('debt_percent' , ascending=False)  
data_pivot

debt,0,1,debt_percent
purpose_category,,,
автомобиль,3905,403,9.354689
образование,3644,370,9.217738
свадьба,2149,186,7.965739
строительство,1735,144,7.663651
недвижимость,7725,603,7.240634
ремонт,572,35,5.766063


Согласно полученным данным сводной таблицы чаще всего задолженность по кредиту возникает у тех заемщиков, который берут кредит для приобретения автомобиля и на образование. Самая низкая задолженность наблюдается у тек, кто берет кредит на ремонт недвижимости. Между ними находятся те, кто кредитуется для проведения свадьбы, строительства и приобретения недвижимости.

**Выводы**

Цель получения кредита влияет на его возврат в срок.

## Итоги исследования

Проведена проверка двух гипотез и установлено:

1. Семейное положение клиента влияет на факт погашения кредита в срок.
2. Наличие детей у клиента влияет на факт погашения кредита в срок.

Первая и вторая гипотезы полностью подтвердились.

Даны ответы на два вопроса:

1. Явная зависимость между уровнем дохода и возвратом кредита в срок прослеживается только в сравнении с очень низким уровнем дохода и иным доходом.
2. Цель получения кредита влияет на его возврат в срок.